# 1. Dataset from scratch

In [6]:
import numpy as np 
import pandas as pd 
import os
import torch
import librosa
from torch.utils.data import Dataset, DataLoader


In [10]:
SAMPLE_RATE=44100
DATA_PATH_NeuroV = 'neurovoz/zenodo_upload/audios/'
labels='labels.csv'

In [16]:
def num_seg(data, SAMPLE_RATE, output_csv):
    time_leng=0.4
    sample_leng=int(time_leng*SAMPLE_RATE)
    overloap=2
    segments_info=[]


    #Processs data to train
    for data_ind, file_path in enumerate(data.Path):
        audio, sample_rate = librosa.load(file_path, sr=SAMPLE_RATE)
        audio_len=len(audio)
        #print(audio_len)
        #plt.plot(audio)
        audio=audio/np.max(abs(audio))
        indx=[i for i,x in enumerate(np.sqrt(abs(audio))) if x>.30]
        segments=0
        if (indx[0]+sample_leng)<audio_len:
            for i in range(int((-indx[0]+indx[len(indx)-1])/(sample_leng/overloap))):
                ind_start = i * int(sample_leng/overloap)+indx[0]
                ind_end = ind_start + sample_leng
                if ind_end <= indx[-1]:
                    segments+=1
            
            segments_info.append({
                'Filename': file_path,
                'Segments': segments,
                
            })
            print(" Processed {}/{} files".format(data_ind,len(data)-1),end='')
        else:
             print(" Processed {}/{} files".format(data_ind,len(data)-1),end='')
             segments_info.append({
                'Filename': file_path,
                'Segments': 0,
                
            })
    

    df_segments = pd.DataFrame(segments_info)

    df_segments.to_csv(output_csv, index=False)

In [12]:
class CustomAudioDataset(Dataset):
    def __init__(self, audio_labels, data_dir, transform=None, target_transform=None):
        self.audio_labels=pd.read_csv(audio_labels) # se obtiene de un script aparte y contiene [ID, label, filename]
        self.data_dir=data_dir
        self.transform=transform
        self.target_transform=target_transform #transformacion de los labels
    def __len__(self):
        return len(self.audio_labels)
    def __getitem__(self, idx):
        data_path=os.path.join(self.data_dir, self.audio_labels[idx, -1]) #directory/path + /filename.wav
        audio, sample_rate = librosa.load(data_path, sr=SAMPLE_RATE)
        label=self.audio_labels[idx, 1]
        subject_group= self.audio_labels[idx, 0]
        if self.transform:
            audio=self.transform(audio)
        if self.target_transform(audio):
            label=self.target_transform(label)
        return audio, subject_group, label

In [13]:
#Dataset
datasetAudio = CustomAudioDataset(labels, DATA_PATH_NeuroV)

#DataLoader
dataloader = DataLoader(datasetAudio, batch_size=32, shuffle=True)

